### To build the procgen
in terminal:
python -c "from procgen.builder import build; build()"

In [1]:
import gym, time, procgen
from wrappers import make_fruitbot_basic

env=gym.make('procgen:procgen-fruitbot-v0', render_mode='human')
env = make_fruitbot_basic(env)
obs=env.reset()
print('viewer started; close the window or press Ctrl+C to stop')
import itertools
print("action space", env.action_space)
for i in range(100):
    action = env.action_space.sample()
    # Handle both gym (4 values) and gymnasium (5 values) API
    obs, rew, done, truncated, info = env.step(action)
    if done: obs = env.reset()
    time.sleep(0.05)
print('done')

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
c:\Users\matan\anaconda3\envs\rl_env\lib\site-packages\gym\utils\passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
c:\Users\matan\anaconda3\envs\rl_env\lib\site-packages\gym\utils\passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initia

viewer started; close the window or press Ctrl+C to stop
action space Discrete(4)
done
done


In [ ]:
# Load a trained PPO model and run it on the existing `env`.
# If the model file is missing, fall back to random actions.
import os
from stable_baselines3 import PPO
import gym, time, procgen
from wrappers import make_fruitbot_basic

model_path = "models/fruitbot/20251117-133701/ppo_final.zip"

# Control which level to test on
START_LEVEL = 0  # Change this to test different levels (0, 1, 2, ...)
NUM_LEVELS = 1   # Set to 1 for single level, 10 for training set, 0 for random

env = gym.make('procgen:procgen-fruitbot-v0', 
               render_mode='human', 
               distribution_mode='easy',
               )
env = make_fruitbot_basic(env)
obs = env.reset()
if os.path.exists(model_path):
    model = PPO.load(model_path, env=env)
    print(f"Loaded PPO model from {model_path}")
else:
    model = None
    print(f"Model not found at {model_path}. Falling back to random actions.")


# Run for a number of steps (or until user closes the viewer)
max_steps = 200
for step in range(max_steps):
    if model is not None:
        action, _states = model.predict(obs, deterministic=True)
    else:
        action = env.action_space.sample()

    result = env.step(action)
    # handle both gym (obs, rew, done, info) and gymnasium (obs, rew, done, truncated, info)
    if len(result) == 5:
        obs, rew, done, truncated, info = result
    else:
        obs, rew, done, info = result
        truncated = False

    if done or truncated:
        obs = env.reset()

    # small delay so human viewer can follow
    time.sleep(0.02)

print("Run finished")

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


building procgen...done


c:\Users\matan\anaconda3\envs\rl_env\lib\site-packages\gym\utils\passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
c:\Users\matan\anaconda3\envs\rl_env\lib\site-packages\gym\utils\passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
c:\Users\matan\anaconda3\envs\rl_env\lib\site-packages\gym\utils\passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(
c:\Users\matan\anaconda3\envs\rl_env\lib\site-packages\gym\utils\passive_env_checker.py:219: Deprecati

Model not found at models/fruitbot/20251117-133701/ppo_final.zip. Falling back to random actions.
Run finished


: 

## CNN architecuter

In [2]:
from stable_baselines3 import PPO
import torch

# Load your trained model
model = PPO.load("models/fruitbot/20251117-143015/ppo_final.zip")

print("="*60)
print("FULL ACTOR-CRITIC NETWORK")
print("="*60)
print(model.policy)


# Count parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("\n" + "="*60)
print("PARAMETER COUNT")
print("="*60)
print(f"Feature extractor: {count_parameters(model.policy.features_extractor):,} params")
print(f"Actor head: {count_parameters(model.policy.action_net):,} params")
print(f"Critic head: {count_parameters(model.policy.value_net):,} params")
print(f"Total: {count_parameters(model.policy):,} params")

FULL ACTOR-CRITIC NETWORK
ActorCriticCnnPolicy(
  (features_extractor): NatureCNN(
    (cnn): Sequential(
      (0): Conv2d(3, 32, kernel_size=(8, 8), stride=(4, 4))
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
      (3): ReLU()
      (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (5): ReLU()
      (6): Flatten(start_dim=1, end_dim=-1)
    )
    (linear): Sequential(
      (0): Linear(in_features=1024, out_features=512, bias=True)
      (1): ReLU()
    )
  )
  (pi_features_extractor): NatureCNN(
    (cnn): Sequential(
      (0): Conv2d(3, 32, kernel_size=(8, 8), stride=(4, 4))
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
      (3): ReLU()
      (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (5): ReLU()
      (6): Flatten(start_dim=1, end_dim=-1)
    )
    (linear): Sequential(
      (0): Linear(in_features=1024, out_features=512, bias=True)
      (1): ReLU()
    )
  )
  (vf_features_extracto